In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json

In [2]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
EXPONENT = 15

class SeparableConv2D(nn.Module):
    '''Separable convolution'''
    def __init__(self, in_channels, out_channels, stride=1):
        super(SeparableConv2D, self).__init__()
        self.dw_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pw_conv =  nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x):
        x = self.dw_conv(x)
        x = self.pw_conv(x)
        return x

In [3]:
input = torch.randn((1, 3, 5, 5))
model = SeparableConv2D(3, 6)

In [4]:
def PointwiseConv2d(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
    kernelSize = 1
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    str_out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[None for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nFilters):
                for k in range(nChannels):
                    out[row][col][filter] += int(input[row*strides, col*strides, k]) * int(weights[k, filter])
                    
                out[row][col][filter] += int(bias[filter])
                remainder[row][col][filter] = str(int(out[row][col][filter] % n))
                out[row][col][filter] = int(out[row][col][filter] // n)
                str_out[row][col][filter] = str(out[row][col][filter] % p)
                            
    return out, str_out, remainder

In [5]:
weights = model.pw_conv.weight.detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()

expected = model.pw_conv(input).detach().numpy()

weights = weights.transpose((2, 3, 1, 0)).squeeze()

quantized_image = input.squeeze().numpy().transpose((1, 2, 0)) * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT
print(f"{quantized_image.shape=}")
print(f"{quantized_weights.shape=}")

actual, str_actual, remainder = PointwiseConv2d(5, 5, 3, 6, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)

actual_scaled = [[[actual[i][j][k] / 10**EXPONENT for k in range(6)] for j in range(5)] for i in range(5)]

expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual_scaled, atol=0.00001))

quantized_image.shape=(5, 5, 3)
quantized_weights.shape=(3, 6)
expected.shape=(1, 6, 5, 5)
expected.shape=(5, 5, 6)


In [6]:
circuit_in = quantized_image.round().astype(int).astype(str).tolist()
circuit_weights = quantized_weights.round().astype(int).astype(str).tolist()
circuit_bias = bias.round().astype(int).astype(str).tolist()

input_json_path = "pointwiseConv2D_input.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": circuit_in,
               "weights": circuit_weights,
               "remainder": remainder,
               "out": str_actual,
               "bias": circuit_bias,
              },
              input_file)